Имеется файл с различными оценками асессоров. 

Формат файла: login tuid docid jud cjud 

Пояснение к формату: login — логин асессора; 
                     uid — id асессора (user id); 
                     docid — id оцениваемого документа (document id); 
                     jud — оценка асессора (judgement); 
                     cjud — правильная оценка (correct judgement); 
                     
                     разделитель — табуляция \t. 
                     
Оценки могут принимать значение [0, 1], т.е. задание, которое сделали асессоры, имеет бинарную шкалу. 
Используя данные об оценках, установите, какие асессоры хуже всего справились с заданием. 
На какие показатели вы ориентировались и какие метрики вы использовали для ответа на этот вопрос? 
Можно ли предложить какие-то новые метрики для подсчета качества асессоров с учетом природы оценок у этого бинарного задания? 
Опишите подробно все этапы вашего решения.


1. сгруппировать данные по асессорам и заданиям
2. для каждого ассесора найти процент правильно выполненных заданий
3. для каждого задания найти процент правильного выполнения 
4. для каждого ассесора вычислить эффективность его работы на 10% самых лёгких и 10% самых сложных заданий
5. создать сводную таблицу
6. проанализировать полученные результаты, сделать выводы о качестве работы асессоров

In [73]:
import pandas as pd
import numpy as np

In [78]:
# Set ipython's max row display
pd.set_option('display.max_row', 30)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [6]:
dataframe = pd.read_csv('data_task3.csv', sep='\t', index_col=0)

In [18]:
dataframe.head(2)

,uid,docid,jud,cjud
login,,,,
assessor158,158,0,0,0
assessor238,238,0,0,0


In [117]:
# 1. сгруппировать данные по асессорам и заданиям

group_by_user = dataframe.groupby('login')      # группировка по асессорам
number_user = len(group_by_user)

group_by_docid = dataframe.groupby('docid')     # группировка по документам
number_doc = len(group_by_docid)

number_notes = len(dataframe)

In [124]:
print('количество асессоров  -  ' + str(number_user) + '\nколичество документов  -  ' + str(number_doc))
print('\nвсего записей  -  ' + str(number_notes) + '\n\nзаписей на одного ассессора в среднем  -  ' + str(number_notes / number_user))
print('записей на один документ  -  ' + str(number_notes / number_doc))

количество асессоров  -  600
количество документов  -  50000

всего записей  -  250000

записей на одного ассессора в среднем  -  416.6666666666667
записей на один документ  -  5.0


In [28]:
dataframe.iloc[0]

uid      158
docid      0
jud        0
cjud       0
Name: assessor158, dtype: int64

In [125]:
# создание глубокой копии фрейма данных

df = dataframe.copy()

In [112]:
# adding new empty columns

df['success'] = np.nan

# создание списка ключей (списка уникальных значений асессоров)

keys = df.index.unique()

In [39]:
df.head(5)

,uid,docid,jud,cjud,success
login,,,,,
assessor158,158,0,0,0,NaN
assessor238,238,0,0,0,NaN
assessor488,488,0,0,0,NaN
assessor136,136,0,0,0,NaN
assessor300,300,0,0,0,NaN


In [46]:
# заполение столбца бинарными маркерами успешности выполнения задания асессором
# 1 - для правильно оцененного документа, 0 - для проваленной оценки

for i in range (0, len(df)):
    if df['jud'].iloc[i] == df['cjud'].iloc[i]:
        df.iloc[i, df.columns.get_loc('success')] = 1
    else:
        df.iloc[i, df.columns.get_loc('success')] = 0

In [49]:
# Контроль корректности заполнения

df.head(10)

,uid,docid,jud,cjud,success
login,,,,,
assessor158,158,0,0,0,1.0
assessor238,238,0,0,0,1.0
assessor488,488,0,0,0,1.0
assessor136,136,0,0,0,1.0
assessor300,300,0,0,0,1.0
assessor123,123,1,1,1,1.0
assessor491,491,1,0,1,0.0
assessor409,409,1,1,1,1.0
assessor396,396,1,1,1,1.0


In [62]:
gbu = df.groupby('login')
gbum = df.groupby('login').mean()      # группировка по асессорам c вычислением среднего

In [92]:
# пример получения отдельной группы по ключу

gbu.get_group('assessor0') 

,uid,docid,jud,cjud,success
login,,,,,
assessor0,0,11,0,0,1.0
assessor0,0,110,1,1,1.0
assessor0,0,147,0,0,1.0
assessor0,0,392,0,0,1.0
assessor0,0,505,0,0,1.0
...,...,...,...,...,...
assessor0,0,49238,0,0,1.0
assessor0,0,49270,0,0,1.0
assessor0,0,49617,0,0,1.0


Отсортировав полученные данные по значению столбца "success" мы получим простейший показатель оценки эффективности ассессора, выраженный в доле правильно выполненных им оценок.
Как видно из полученных данных, эффективность ассессоров в верхнем и нижнем топ-5 отличается, практически, в 2 раза.

Однако, данного показаетеля недостаточно для корректной оценки эффективности работы ассессора, так как мы не учитываем различия в сложности оценок каждого отдельного документа. Необходимость учета данной особенности исходит из того, что каждый документ не был оценён каждым из ассессоров, а также в условиях не указано, что оценка каждого документа равно трудозатратна.

Так как мы не можем дополнить исходные данные для соответсвия выдвинотому условию, необходимо произвести оценку эффективности работы асессоров на 10% статистически наиболее сложных и 10% статистически наиболее лёгких документов.

In [114]:
gbum.sort_values(by = ['success'], ascending=False)

,uid,docid,jud,cjud,success
login,,,,,
assessor296,296.0,24049.261905,0.154762,0.107143,0.923810
assessor545,545.0,25293.725225,0.162162,0.130631,0.923423
assessor184,184.0,24549.223881,0.171642,0.109453,0.922886
assessor63,63.0,24189.748166,0.161369,0.107579,0.921760
assessor191,191.0,25657.260331,0.159091,0.101240,0.921488
...,...,...,...,...,...
assessor234,234.0,26636.383838,0.535354,0.141414,0.484848
assessor390,390.0,25813.803398,0.541262,0.114078,0.480583
assessor118,118.0,23988.383632,0.547315,0.099744,0.475703
